## Define a new Function
OpenAI chat completion may pick between generating a reply message, or a function call.
This provides an assistant with the ability to define a new function that its model may call.

In [3]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import requests
import os

config_list = config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # Function calling with GPT 3.5
        "model": ["gpt-3.5-turbo-16k-0613"],
    },
)
llm_config = {
    "functions": [
        {
            "name": "define_function",
            "description": "Define a function that can be invoked and re-used.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the function to define."
                    },
                    "code": {
                        "type": "string",
                        "description": "The Python code that implements the function."
                    }
                },
                "required": ["name", "code"],
            },
        },
    ],
    "config_list": config_list,
    "request_timeout": 120,
}

def _is_termination_msg(message):
    """Check if a message is a termination message."""
    if isinstance(message, dict):
        message = message.get("content")
        if message is None:
            return False
        return message.rstrip().endswith("TERMINATE")

assistant = AssistantAgent(
    name="chatbot",
    system_message="""You are an assistant.
        The user will ask a question.
        You may use the provided functions before providing a final answer.
        Only use the functions you were provided.
        When the answer has been provided, reply TERMINATE.""",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": True},
    is_termination_msg=_is_termination_msg,
    default_auto_reply="Reply TERMINATE when the initial request has been fulfilled.",
    human_input_mode="NEVER")

def greet():
    return "hello"

from IPython import get_ipython
def define_function(name, code):
    function_config = {
        "name": name,
        "description": "New function",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }
    llm_config["functions"] = llm_config["functions"] + [function_config]
    user_proxy.register_function(
        function_map={
            "name": greet
        }
    )
    return f"Function {name} has been defined:\n{code}"

user_proxy.register_function(
    function_map={
        "define_function": define_function
    }
)

user_proxy.initiate_chat(
    assistant, message="What functions do you know about?")

user_proxy.initiate_chat(
    assistant, message="Define a function that says hello.")

user_proxy.initiate_chat(
    assistant, message="What functions do you know about, now?")


user_proxy (to chatbot):

What functions do you know about?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

I know about the following functions:

1. define_function: This function allows you to define new functions that can be invoked and re-used.

Please let me know if you need any further assistance.

--------------------------------------------------------------------------------
user_proxy (to chatbot):

Reply TERMINATE when the initial request has been fulfilled.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
user_proxy (to chatbot):

Define a function that says hello.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: define_function *****
Arguments: 
{
  "name": "say_hello",
  "code